# This is the processsing of the protest data. 

In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta

### Shapefiles.

In [2]:
districts = []
# getting the shape file for the provinces
dist2 = gpd.read_file('DATA\ZA_SHAPES\zaf_adm_sadb_ocha_20201109_SHP\zaf_admbnda_adm1_sadb_ocha_20201109.shp')
dist2 = dist2[['Shape_Area','ADM1_EN','geometry']]
dist2.rename(columns={'ADM1_EN':'Place0'}, inplace=True, errors='raise')
districts.append(dist2)

# geting the shape file for municipalities 
dist3 = gpd.read_file('DATA\ZA_SHAPES\zaf_adm_sadb_ocha_20201109_SHP\zaf_admbnda_adm2_sadb_ocha_20201109.shp')
dist3 = dist3[['Shape_Area','ADM2_EN','geometry']]
dist3.rename(columns={'ADM2_EN':'Place1'}, inplace=True, errors='raise')
districts.append(dist3)

# geting the shape file for districs 
dist4 = gpd.read_file('DATA\ZA_SHAPES\zaf_adm_sadb_ocha_20201109_SHP\zaf_admbnda_adm3_sadb_ocha_20201109.shp')
dist4 = dist4[['Shape_Area','ADM3_EN','geometry']]
dist4.rename(columns={'ADM3_EN':'Place2'}, inplace=True, errors='raise')
districts.append(dist4)

# geting the shape file for districs21 
dist5 = gpd.read_file('DATA\ZA_SHAPES\zaf_adm_sadb_ocha_20201109_SHP\zaf_admbnda_adm4_sadb_ocha_20201109.shp')
dist5 = dist5[['Shape_Area','ADM4_EN','geometry']]
dist5.rename(columns={'ADM4_EN':'Place3'}, inplace=True, errors='raise')
dist5 = dist5.astype({'Place3':int})
districts.append(dist5)

In [3]:
def adding_point(protests):
    #creating a geometry for each of the protests
    # Dropping all the Na
    protests = protests[protests['longitude'].notna()]
    protests = protests[protests['latitude'].notna()]


    protests["latitude"] = pd.to_numeric(protests["latitude"],errors='coerce')
    protests["longitude"] = pd.to_numeric(protests["longitude"],errors='coerce')

    protests = protests[protests['longitude'].notna()]
    protests = protests[protests['latitude'].notna()]

    protests = gpd.GeoDataFrame(protests, geometry=gpd.points_from_xy(protests.longitude, protests.latitude))
    protests = protests.drop(['latitude','longitude'],axis=1)
    return(protests)

In [4]:
def Adding_All_Locations(districts, csv):
    n = 0
    for dist in districts:
        points = csv['geometry']
        shapes = dist['geometry']
        places = [0]*len(csv)

        # we are going to need to enumerate
        for i, point in enumerate(points):
            temp = shapes.contains(point)
            temp = temp.tolist()
            if True in temp:
                loc = temp.index(True)
                place = dist.iloc[loc,1]
                places[i] = place
        
        csv['Place'+ str(n)] = places
        csv.reset_index()
        n=n+1

### Tweets

In [5]:
tweets = pd.read_csv('DATA/rawtweets.csv')
tweets['geometry'] = gpd.GeoSeries.from_wkt(tweets['geometry'])
tweets = tweets.drop(['Unnamed: 0'],axis=1)

In [6]:
# Adding the locations to the tweets
# around 18 mins
Adding_All_Locations(districts, tweets)
tweets.to_csv('DATA/tweets.csv')


### Protests

In [9]:
df = pd.read_csv('DATA/fmf.csv')
protests = df[['event_date','notes','latitude','longitude']]
protests = protests.rename(columns = {'event_date': 'Date', 'admin3': 'Place'})
protests['Date']=pd.to_datetime(protests['Date'], format='%Y %m %d')
protests = adding_point(protests)
# 2017-10-25

In [10]:
# Adding the locations to the protests
Adding_All_Locations(districts, protests)
protests.to_csv('DATA/protests.csv')

### Quick load of Data

In [11]:
tweets = pd.read_csv('DATA/tweets.csv')
tweets['Date']=pd.to_datetime(tweets['Date'], format='%Y %m %d')
tweets['geometry'] = gpd.GeoSeries.from_wkt(tweets['geometry'])
tweets = tweets.drop(['Unnamed: 0'],axis=1)

protests = pd.read_csv('DATA/protests.csv')
protests['Date']=pd.to_datetime(protests['Date'], format='%Y %m %d')
protests['geometry'] = gpd.GeoSeries.from_wkt(protests['geometry'])
protests = protests.drop(['Unnamed: 0'],axis=1)

In [12]:
tweets.head(2)

,text,Date,geometry,author_followers,retweets,replies,likes,quote_count,Subjectivity,Polarity,...,locations,weapons,eventualities,curiosities,non_protests,universities,Place0,Place1,Place2,Place3
0,@ItIsSeRomeo @maggsnaidu @Julius_S_Malema My q...,2017-12-30,POINT (27.96810 -26.27614),1351,1,3,11,0,0.485714,0.307143,...,['Tertiary Edu'],NaN,NaN,NaN,NaN,NaN,Gauteng,City of Johannesburg,City of Johannesburg,125
1,@AndileDakuse @eNCA In 1993 was admitted in 3 ...,2017-12-30,POINT (28.38669 -25.71081),813,0,0,1,0,0.623810,0.371429,...,NaN,NaN,NaN,NaN,['Football matches'],"['University of Pretoria', 'Tshwane University...",Gauteng,City of Tshwane,City of Tshwane,18


In [13]:
protests.head(2)

,Date,notes,geometry,Place0,Place1,Place2,Place3
0,2017-10-25,A few hundred Fees Must Fall protesters attemp...,POINT (18.47640 -33.96330),Western Cape,City of Cape Town,City of Cape Town,58
1,2017-09-19,A group of #FeesMustFall students has stormed ...,POINT (18.47640 -33.96330),Western Cape,City of Cape Town,City of Cape Town,58
